# K-Nearest Neighbors and Centroid Methods

In [61]:
import random
import operator
import pandas as pd
import numpy as np
import nltk
from collections import Counter
from matplotlib import pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.corpus import words
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import math
import time
import sys
from scipy import sparse
%matplotlib inline

### Useful functions (normalization, score ...)

In [74]:
def get_mails_ca_is_recipient(training_info, ca):
    return training_info[training_info.recipients.map(lambda mail_list: ca in mail_list.split(' '))].mid.tolist()

def get_idx_mails_ca_is_recipient(training_info, ca):
    return training_info.recipient_list.map(lambda mail_list: ca in mail_list)

def normalize(text, tokenize=True, stemmer=False):
    porter = nltk.PorterStemmer()
    tokenized_text = nltk.word_tokenize(text) if tokenize else text
    if stemmer:
        normalized_text = [porter.stem(w.lower()) for w in tokenized_text]
    else:
        normalized_text = [w.lower() for w in tokenized_text]
    return normalized_text

def filter_vocab(normalized_text, normalized_vocab):
    return [w for w in normalized_text if w in normalized_vocab]

def is_email_valid(email):
    return True if re.match(r"(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$)", email) else False

def get_normalized_body(body_set, filter_english=False, stemmer=False):
    if filter_english:
        vocab_english_normalized = set(normalize(words.words('en'), tokenize=False, stemmer=stemmer))
        return body_set.map(lambda x: filter_vocab(normalize(x, stemmer=stemmer), vocab_english_normalized))
    return body_set.map(lambda x: normalize(x, stemmer=stemmer))

def get_train_test(df, train_size=0.8):
    set_size = df.shape[0]
    permutation = np.random.permutation(set_size)
    train_idx = permutation[:int(set_size*train_size)]
    test_idx = permutation[int(set_size*train_size):]
    return df.loc[train_idx], df.loc[test_idx]

def fix_mail_list(recipients):
    recipients = recipients.split(' ')
    return [email for email in recipients if is_email_valid(email)]

def get_score(tfidf_mail, centroid_ca):
    return np.dot(np.array(centroid_ca)[0], np.array(tfidf_mail)[0])/\
                (np.linalg.norm(tfidf_mail)*np.linalg.norm(centroid_ca))

def get_n_best_recipient(tfidf_q, centroid_map, candidates, n=10):
    out = {}
    for ca in candidates:
        score = get_score(tfidf_q, centroid_map[ca].todense())
        if not np.isnan(score):
            out[ca] = score
    out_sorted = sorted(out.items(), key=lambda x: x[1], reverse = True)
    return [d[0] for d in out_sorted[:10]]
    
def apk(actual, predicted, k=10):
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

In [63]:
path_to_data = './'

In [64]:
##########################
# load some of the files #                           
##########################

training = pd.read_csv(path_to_data + 'training_set.csv', sep=',', header=0)
training_info = pd.read_csv(path_to_data + 'training_info.csv', sep=',', header=0)
test = pd.read_csv(path_to_data + 'test_set.csv', sep=',', header=0)
test_info = pd.read_csv(path_to_data + 'test_info.csv', sep=',', header=0)

In [65]:
training.head()

,sender,mids
0,karen.buckley@enron.com,158713 158697 200301 158679 278595 298162 2002...
1,amr.ibrahim@enron.com,215241 3437 215640 3506 191790 3517 3520 3562 ...
2,andrea.ring@enron.com,270705 270706 270707 270708 270709 270710 2707...
3,sylvia.hu@enron.com,111444 111422 183084 111412 111347 110883 1105...
4,phillip.platter@enron.com,327074 327384 327385 264443 274124 274125 2741...


In [66]:
training_info.head()

,mid,date,body,recipients
0,60,2000-07-25 08:14:00,Legal has been assessing the risks of doing bl...,robert.badeer@enron.com murray.o neil@enron.co...
1,66,2000-08-03 02:56:00,Attached is a spreadsheet to estimate export f...,kim.ward@enron.com robert.badeer@enron.com mur...
2,74,2000-08-15 05:37:00,Kevin/Bob: Here is a quick rundown on the cons...,robert.badeer@enron.com john.massey@enron.com ...
3,80,2000-08-20 14:12:00,check this out and let everyone know what s up...,robert.badeer@enron.com jeff.richter@enron.com
4,83,2000-08-22 08:17:00,Further to your letter to us (addressed to Mr....,pgillman@schiffhardin.com kamarlantes@calpx.co...


In [67]:
test.head()

,sender,mids
0,karen.buckley@enron.com,298389 332383 298390 284071 366982 81773 81791...
1,amr.ibrahim@enron.com,48260 48465 50344 48268 50330 48237 189979 189...
2,andrea.ring@enron.com,366364 271168 271172 271167 271189
3,sylvia.hu@enron.com,134931 134856 233549 233517 134895 233584 3736...
4,phillip.platter@enron.com,274220 274225 274215 274223 274214 274207 2742...


In [68]:
test_info.head()

,mid,date,body
0,1577,2001-11-19 06:59:51,Note: Stocks of heating oil are very high for...
1,1750,2002-03-05 08:46:57,"Kevin Hyatt and I are going for ""sghetti"" at S..."
2,1916,2002-02-13 14:17:39,This was forwarded to me and it is funny. - Wi...
3,2094,2002-01-22 11:33:56,I will be in to and happy to assist too. I ma...
4,2205,2002-01-11 07:12:19,Thanks. I needed a morning chuckle.


In [69]:
training_info.shape

(43613, 4)

In [70]:
test_info.shape

(2362, 3)

### Exploration and Feature Engineering

In [71]:
################################
# create some handy structures #                    
################################
                            
# convert training set to dictionary
emails_ids_per_sender = {}
for index, series in training.iterrows():
    row = series.tolist()
    sender = row[0]
    ids = row[1:][0].split(' ')
    emails_ids_per_sender[sender] = ids

# convert training set to dictionary
emails_ids_per_sender_test = {}
for index, series in test.iterrows():
    row = series.tolist()
    sender = row[0]
    ids = row[1:][0].split(' ')
    emails_ids_per_sender_test[sender] = ids
    
# save all unique sender names
all_senders = emails_ids_per_sender.keys()

# create address book with frequency information for each user
address_books = {}
i = 0

for sender, ids in emails_ids_per_sender.items():
    recs_temp = []
    for my_id in ids:
        recipients = training_info[training_info['mid']==int(my_id)]['recipients'].tolist()
        recipients = recipients[0].split(' ')
        # keep only legitimate email addresses
        recipients = [rec for rec in recipients if '@' in rec]
        recs_temp.append(recipients)
    # flatten    
    recs_temp = [elt for sublist in recs_temp for elt in sublist]
    # compute recipient counts
    rec_occ = dict(Counter(recs_temp))
    # order by frequency
    sorted_rec_occ = sorted(rec_occ.items(), key=operator.itemgetter(1), reverse = True)
    # save
    address_books[sender] = sorted_rec_occ
    
    if i % 10 == 0:
        print(i)
    i += 1

0
10
20
30
40
50
60
70
80
90
100
110
120


In [72]:
sender_per_mid = {}
for key in emails_ids_per_sender_test.keys():
    mids = emails_ids_per_sender_test[key]
    for mid in mids:
        sender_per_mid[mid] = key

In [1]:
#sender_per_mid

In [2]:
#emails_ids_per_sender

In [92]:
training_info['normalized_body'] = get_normalized_body(training_info.body, stemmer=True)
test_info['normalized_body'] = get_normalized_body(test_info.body, stemmer=True)

In [93]:
training_info.head()

,mid,date,body,recipients,normalized_body,body_text
0,60,2000-07-25 08:14:00,Legal has been assessing the risks of doing bl...,robert.badeer@enron.com murray.o neil@enron.co...,"[legal, ha, been, assess, the, risk, of, do, b...",legal ha been assess the risk of do block forw...
1,66,2000-08-03 02:56:00,Attached is a spreadsheet to estimate export f...,kim.ward@enron.com robert.badeer@enron.com mur...,"[attach, is, a, spreadsheet, to, estim, export...",attach is a spreadsheet to estim export fee .
2,74,2000-08-15 05:37:00,Kevin/Bob: Here is a quick rundown on the cons...,robert.badeer@enron.com john.massey@enron.com ...,"[kevin/bob, :, here, is, a, quick, rundown, on...",kevin/bob : here is a quick rundown on the con...
3,80,2000-08-20 14:12:00,check this out and let everyone know what s up...,robert.badeer@enron.com jeff.richter@enron.com,"[check, thi, out, and, let, everyon, know, wha...",check thi out and let everyon know what s up. ...
4,83,2000-08-22 08:17:00,Further to your letter to us (addressed to Mr....,pgillman@schiffhardin.com kamarlantes@calpx.co...,"[further, to, your, letter, to, us, (, address...",further to your letter to us ( address to mr. ...


In [94]:
test_info.head()

,mid,date,body,normalized_body,body_text
0,1577,2001-11-19 06:59:51,Note: Stocks of heating oil are very high for...,"[note, :, stock, of, heat, oil, are, veri, hig...",note : stock of heat oil are veri high for thi...
1,1750,2002-03-05 08:46:57,"Kevin Hyatt and I are going for ""sghetti"" at S...","[kevin, hyatt, and, i, are, go, for, ``, sghet...",kevin hyatt and i are go for `` sghetti '' at ...
2,1916,2002-02-13 14:17:39,This was forwarded to me and it is funny. - Wi...,"[thi, wa, forward, to, me, and, it, is, funni,...",thi wa forward to me and it is funni . - witci...
3,2094,2002-01-22 11:33:56,I will be in to and happy to assist too. I ma...,"[i, will, be, in, to, and, happi, to, assist, ...",i will be in to and happi to assist too . i ma...
4,2205,2002-01-11 07:12:19,Thanks. I needed a morning chuckle.,"[thank, ., i, need, a, morn, chuckl, .]",thank . i need a morn chuckl .


In [95]:
training_info['body_text'] = training_info.normalized_body.map(lambda x: ' '.join(x))
test_info['body_text'] = test_info.normalized_body.map(lambda x: ' '.join(x))

In [96]:
training_info.head()

,mid,date,body,recipients,normalized_body,body_text
0,60,2000-07-25 08:14:00,Legal has been assessing the risks of doing bl...,robert.badeer@enron.com murray.o neil@enron.co...,"[legal, ha, been, assess, the, risk, of, do, b...",legal ha been assess the risk of do block forw...
1,66,2000-08-03 02:56:00,Attached is a spreadsheet to estimate export f...,kim.ward@enron.com robert.badeer@enron.com mur...,"[attach, is, a, spreadsheet, to, estim, export...",attach is a spreadsheet to estim export fee .
2,74,2000-08-15 05:37:00,Kevin/Bob: Here is a quick rundown on the cons...,robert.badeer@enron.com john.massey@enron.com ...,"[kevin/bob, :, here, is, a, quick, rundown, on...",kevin/bob : here is a quick rundown on the con...
3,80,2000-08-20 14:12:00,check this out and let everyone know what s up...,robert.badeer@enron.com jeff.richter@enron.com,"[check, thi, out, and, let, everyon, know, wha...",check thi out and let everyon know what s up. ...
4,83,2000-08-22 08:17:00,Further to your letter to us (addressed to Mr....,pgillman@schiffhardin.com kamarlantes@calpx.co...,"[further, to, your, letter, to, us, (, address...",further to your letter to us ( address to mr. ...


In [97]:
test_info.head()

,mid,date,body,normalized_body,body_text
0,1577,2001-11-19 06:59:51,Note: Stocks of heating oil are very high for...,"[note, :, stock, of, heat, oil, are, veri, hig...",note : stock of heat oil are veri high for thi...
1,1750,2002-03-05 08:46:57,"Kevin Hyatt and I are going for ""sghetti"" at S...","[kevin, hyatt, and, i, are, go, for, ``, sghet...",kevin hyatt and i are go for `` sghetti '' at ...
2,1916,2002-02-13 14:17:39,This was forwarded to me and it is funny. - Wi...,"[thi, wa, forward, to, me, and, it, is, funni,...",thi wa forward to me and it is funni . - witci...
3,2094,2002-01-22 11:33:56,I will be in to and happy to assist too. I ma...,"[i, will, be, in, to, and, happi, to, assist, ...",i will be in to and happi to assist too . i ma...
4,2205,2002-01-11 07:12:19,Thanks. I needed a morning chuckle.,"[thank, ., i, need, a, morn, chuckl, .]",thank . i need a morn chuckl .


#### TF-IDF

In [106]:
corpus = training_info.body_text
vectorizer = TfidfVectorizer(min_df=2, max_df=0.005)
tfidf = vectorizer.fit_transform(corpus)
tfidf_test = vectorizer.transform(test_info.body_text)

In [107]:
tfidf

<43613x83000 sparse matrix of type '<class 'numpy.float64'>'
	with 953074 stored elements in Compressed Sparse Row format>

In [21]:
training_info['recipient_list'] = training_info.recipients.map(lambda x: fix_mail_list(x))

In [22]:
training_info.head()

,mid,date,body,recipients,normalized_body,body_text,recipient_list
0,60,2000-07-25 08:14:00,Legal has been assessing the risks of doing bl...,robert.badeer@enron.com murray.o neil@enron.co...,"[legal, has, been, assessing, the, risks, of, ...",legal has been assessing the risks of doing bl...,"[robert.badeer@enron.com, murray.oneil@enron.c..."
1,66,2000-08-03 02:56:00,Attached is a spreadsheet to estimate export f...,kim.ward@enron.com robert.badeer@enron.com mur...,"[attached, is, a, spreadsheet, to, estimate, e...",attached is a spreadsheet to estimate export f...,"[kim.ward@enron.com, robert.badeer@enron.com, ..."
2,74,2000-08-15 05:37:00,Kevin/Bob: Here is a quick rundown on the cons...,robert.badeer@enron.com john.massey@enron.com ...,"[kevin/bob, :, here, is, a, quick, rundown, on...",kevin/bob : here is a quick rundown on the con...,"[robert.badeer@enron.com, john.massey@enron.co..."
3,80,2000-08-20 14:12:00,check this out and let everyone know what s up...,robert.badeer@enron.com jeff.richter@enron.com,"[check, this, out, and, let, everyone, know, w...",check this out and let everyone know what s up...,"[robert.badeer@enron.com, jeff.richter@enron.com]"
4,83,2000-08-22 08:17:00,Further to your letter to us (addressed to Mr....,pgillman@schiffhardin.com kamarlantes@calpx.co...,"[further, to, your, letter, to, us, (, address...",further to your letter to us ( addressed to mr...,"[pgillman@schiffhardin.com, kamarlantes@calpx...."


In [23]:
all_recipients = np.unique(np.concatenate(training_info.recipient_list))

In [135]:
train, test = get_train_test(training_info)
all_recipients = np.unique(np.concatenate(train.recipient_list.tolist()))

#### Stemming

In [136]:
porter = nltk.PorterStemmer()
#vectorizer = TfidfVectorizer(min_df=2, stop_words=[porter.stem(w.lower()) for w in stopwords.words('english')])
vectorizer = TfidfVectorizer(min_df=5, stop_words=[w.lower() for w in stopwords.words('english')])
tfidf_train_ = vectorizer.fit_transform(train.body_text)
tfidf_test_ = vectorizer.transform(test.body_text)

In [137]:
tfidf_train_.shape

(34890, 34659)

In [138]:
train['recipient_list'] = train.recipients.map(lambda x: fix_mail_list(x))
test['recipient_list'] = test.recipients.map(lambda x: fix_mail_list(x))

In [139]:
#sum_ca = sum(tfidf_train_.todense())

### 1/ Centroid Method

In [140]:
def centroid(ca):
    #alpha = 16
    #beta = 4
    #sent_train = tfidf_train_.shape[0]
    D_ca = get_idx_mails_ca_is_recipient(train, ca)
    n_D_ca = sum(D_ca)
    sum_D_ca = sum(tfidf_train_[np.argwhere(D_ca == True).flatten(),].todense())
    #sum_not_D_ca = sum_ca - sum_D_ca
    #return (alpha/n_D_ca)*sum_D_ca + (beta/(sent_train-n_D_ca))*sum_not_D_ca
    return (1/n_D_ca)*sum_D_ca

In [141]:
centroid_map = {}
for index, mail in enumerate(all_recipients):
    if index % 20 == 0:
        sys.stdout.write("\r"+str(math.ceil(100*index/len(all_recipients)))+"% done")
        sys.stdout.flush()
    centroid_map[mail] = sparse.coo_matrix(centroid(mail))

100% done

In [3]:
"""
### On test set

s = 0
recipient_list_ = test.recipient_list.tolist()
for i in range(test.shape[0]):
    score = apk(recipient_list_[i], get_n_best_recipient(tfidf_test_[i].todense(), centroid_map, all_recipients))
    s += score
    print(i+1, s/(i+1), score)
"""  

'\n### On test set\n\ns = 0\nrecipient_list_ = test.recipient_list.tolist()\nfor i in range(test.shape[0]):\n    score = apk(recipient_list_[i], get_n_best_recipient(tfidf_test_[i].todense(), centroid_map, all_recipients))\n    s += score\n    print(i+1, s/(i+1), score)\n'

In [1]:
### On train set

#s = 0
#recipient_list_ = train.recipient_list.tolist()
#for i in range(train.shape[0]):
#    score = apk(recipient_list_[i], get_n_best_recipient(tfidf_train_[i].todense(), centroid_map, all_recipients))
#    s += score
#    print(i+1, s/(i+1), score)

### 2/K-Nearest Neighbors Method

In [25]:
def get_inv_norm_sparse_matrix(sm, fraction=20):
    out = []
    n_components = sm.shape[0]
    block_size = math.floor(n_components/fraction)
    for i in range(fraction+1):
        sys.stdout.write("\r"+str(math.ceil(100*i/fraction))+"% done")
        sys.stdout.flush()
        sm_block = sm[i*block_size:min((i+1)*block_size, n_components)]
        sm_block_norm = list(sm_block.dot(sm_block.T).dot(sparse.identity(sm_block.shape[0])).dot(np.ones(sm_block.shape[0])))
        sm_block_norm = [1/math.sqrt(x) if x != 0 else np.inf for x in sm_block_norm]
        out += list(sm_block_norm)
    return out

In [26]:
idtdf_inv_norm = np.array(get_inv_norm_sparse_matrix(tfidf))

100% done

In [27]:
idtdf_test_inv_norm = np.array(get_inv_norm_sparse_matrix(tfidf_test))

100% done

In [28]:
cosine_not_normalized = tfidf.dot(tfidf_test.T)
cosine_not_normalized.shape

(43613, 2362)

In [29]:
inv_norm_product = idtdf_inv_norm.reshape(len(idtdf_inv_norm),1).dot(idtdf_test_inv_norm.reshape(1,len(idtdf_test_inv_norm)))
inv_norm_product.shape

(43613, 2362)

In [30]:
cosine_normalized = np.array(cosine_not_normalized.todense())*inv_norm_product
cosine_normalized.shape

(43613, 2362)

In [31]:
cosine_normalized_sorted_arg = np.array(np.argsort(-cosine_normalized, axis=0))
cosine_normalized_sorted_arg.shape

(43613, 2362)

In [32]:
most_similar_docs = []
docs_in_test = cosine_normalized_sorted_arg.shape[1]
for idx in range(docs_in_test):
    sys.stdout.write("\r"+str(math.ceil(100*idx/docs_in_test))+"% done")
    sys.stdout.flush()
    most_similar_docs.append([y[1] for y in sorted([(x, index) for (x, index) in enumerate(cosine_normalized_sorted_arg[:,idx])])[:30]])

100% done

In [33]:
most_similar_docs = np.array(most_similar_docs)
most_similar_docs.shape

(2362, 30)

In [34]:
def get_score(ca, idx_test, most_similar_docs=most_similar_docs, df=training_info, cosine_normalized=cosine_normalized):
    return np.sum(np.array([1 if ca in x else 0 for x in df.recipient_list.loc[most_similar_docs[idx_test]].values])*np.array([cosine_normalized[idx_train, idx_test] for idx_train in most_similar_docs[idx_test]]))

In [57]:
def get_best_n_recipient(idx_test):
    mid = test_info.mid[idx_test]
    sender = sender_per_mid[str(mid)]
    recipients_potential = [x[0] for x in address_books[sender]]
    out = {}
    for index, ca in enumerate(all_recipients):
        #sys.stdout.write("\r"+str(math.ceil(100*index/len(all_recipients)))+"% done")
        #sys.stdout.flush()
        out[ca] = get_score(ca, idx_test)
    return [y[0] for y in sorted(out.items(), key=lambda x: x[1], reverse=True) if y[0] in recipients_potential][:10]

### Tests

In [59]:
out_final = []
path_to_results = './'
for i in range(test_info.shape[0]):
    out_final.append(get_best_n_recipient(i))
    if i % 10 == 0:
        print(i)
        with open(path_to_results + 'res_knn.txt', 'w') as my_file:
            for index, my_preds in enumerate(out_final):
                my_file.write(str(index) + ',' + ' '.join(my_preds) + '\n')

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

In [60]:
path_to_results = './'
with open(path_to_results + 'predictions_knn.txt', 'w') as my_file:
    my_file.write('mid,recipients' + '\n')
    for mid, my_preds in zip(test_info.mid, out_final):
        my_file.write(str(mid) + ',' + ' '.join(my_preds) + '\n')

As a result on the public leaderboard, we get a MAP@10 of 0.31 for centroids and 0.27 for K-Nearest neighbors.